# 拆书系统 - 知识图谱 Worker 节点 (Kaggle)

## 说明

这是一个在 Kaggle 上运行的知识图谱 Worker 节点，适用于**私有代码仓库**。

### 环境要求
- **CPU Only** 环境即可（不需要 GPU）
- **Internet** 必须开启
- **资源**: 4核 CPU + 16GB 内存

### 运行时长
- 免费版: 最长 9 小时，每周 30 小时配额
- 付费版: 无限时长

### 配置步骤

#### 1. 准备代码包（本地执行）
```bash
cd chaishu-vue3
zip -r chaishu-vue3.zip . -x "*.git*" "*node_modules*" "*frontend/dist*" "*__pycache__*" "*.pyc" "*venv*"
```

#### 2. 上传到 Kaggle（选择其一）
- **方式一**: 在 Notebook 右侧点击 **+ Add Data** → **Upload** → 上传 `chaishu-vue3.zip`
- **方式二**: 创建私有 Dataset，然后在 Notebook 中添加

#### 3. 配置 Kaggle 环境
- **Accelerator**: CPU (不要选 GPU)
- **Internet**: ON（必需！）
- **Persistence**: OFF（可选）

#### 4. 修改配置并运行
- 修改下方 **Cell 3: 配置环境变量** 中的连接信息
- 依次运行所有单元格

---

## 1. 安装依赖包

In [ ]:
%%time
print("开始安装依赖包...")

!pip install -q Flask==2.3.3 \
    Flask-CORS==4.0.0 \
    Flask-SocketIO==5.3.6 \
    SQLAlchemy==2.0.21 \
    httpx==0.25.0 \
    PyMySQL==1.1.0 \
    neo4j==5.28.0 \
    redis==5.0.0 \
    python-dotenv==1.0.0 \
    APScheduler==3.11.0 \
    requests==2.32.0 \
    chardet==5.2.0

print("✅ 依赖包安装完成")

## 2. 上传项目代码

**⚠️ 重要：需要手动上传项目代码包**

### 方式一：上传 ZIP 包（推荐）
1. 在本地将项目打包为 `chaishu-vue3.zip`
2. 在 Kaggle Notebook 右侧点击 **Add Data** → **Upload**
3. 上传 ZIP 文件
4. 运行下面的代码解压

### 方式二：使用 Kaggle Dataset
1. 将项目代码上传为私有 Dataset
2. 在 Notebook 中添加该 Dataset
3. 代码会自动从 `/kaggle/input/` 读取

In [ ]:
%%time
import os
import zipfile
from pathlib import Path

# 方式一：从上传的 ZIP 解压
zip_path = '/kaggle/working/chaishu-vue3.zip'
dataset_path = '/kaggle/input/chaishu-vue3'

if os.path.exists(zip_path):
    print("检测到上传的 ZIP 文件，开始解压...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('/kaggle/working/')
    print("✅ ZIP 解压完成")
    
    # 查找解压后的目录（可能在子目录中）
    if os.path.exists('/kaggle/working/chaishu-vue3'):
        project_dir = '/kaggle/working/chaishu-vue3'
    else:
        # 查找第一个包含 worker.py 的目录
        for root, dirs, files in os.walk('/kaggle/working'):
            if 'worker.py' in files:
                project_dir = root
                break
        else:
            raise FileNotFoundError("未找到项目目录，请确认 ZIP 包结构")
    
# 方式二：从 Kaggle Dataset 复制
elif os.path.exists(dataset_path):
    print("检测到 Kaggle Dataset，开始复制...")
    import shutil
    project_dir = '/kaggle/working/chaishu-vue3'
    shutil.copytree(dataset_path, project_dir)
    print("✅ Dataset 复制完成")
    
else:
    raise FileNotFoundError(
        "❌ 未找到项目代码！\n\n"
        "请选择以下方式之一上传代码：\n"
        "1. 上传 chaishu-vue3.zip 到 /kaggle/working/\n"
        "2. 添加 chaishu-vue3 Dataset 到此 Notebook"
    )

# 切换到项目目录
%cd {project_dir}

print(f"✅ 代码准备完成")
print(f"当前目录: {os.getcwd()}")

# 验证关键文件
required_files = ['worker.py', 'requirements.txt', 'src/services/knowledge_graph_service.py']
missing_files = [f for f in required_files if not os.path.exists(f)]
if missing_files:
    print(f"⚠️ 警告：缺少文件 {missing_files}")
else:
    print("✅ 关键文件检查通过")

## 3. 配置环境变量

**⚠️ 重要: 请修改下方的配置信息为你的实际服务器地址和密码**

In [ ]:
import os

# ==================== Redis 配置 ====================
os.environ['REDIS_HOST'] = 'your-redis-host.com'        # 修改为你的 Redis 地址
os.environ['REDIS_PORT'] = '6379'
os.environ['REDIS_PASSWORD'] = 'your-redis-password'    # 修改为你的 Redis 密码
os.environ['REDIS_DB'] = '0'

# ==================== MySQL 配置 ====================
os.environ['DB_HOST'] = 'your-mysql-host.com'           # 修改为你的 MySQL 地址
os.environ['DB_PORT'] = '3306'
os.environ['DB_USER'] = 'root'                          # 修改为你的 MySQL 用户名
os.environ['DB_PASSWORD'] = 'your-mysql-password'       # 修改为你的 MySQL 密码
os.environ['DB_NAME'] = 'chaishu'

# ==================== Neo4j 配置 ====================
os.environ['NEO4J_URI'] = 'bolt://your-neo4j-host.com:7687'  # 修改为你的 Neo4j 地址
os.environ['NEO4J_USER'] = 'neo4j'
os.environ['NEO4J_PASSWORD'] = 'your-neo4j-password'    # 修改为你的 Neo4j 密码

# ==================== Worker 配置 ====================
# Kaggle CPU 环境: 4核 16GB 内存，可以运行较多进程
os.environ['KG_WORKERS_PER_PROVIDER'] = '8'             # 每个 Provider 8 个进程
os.environ['KG_WORKER_NODE_NAME'] = 'kaggle-cpu-worker-1'
os.environ['KG_MAX_TOTAL_PROCESSES'] = '50'
os.environ['KG_MAX_PROCESSES_PER_PROVIDER'] = '10'
os.environ['LOG_LEVEL'] = 'INFO'

# 设置 Docker 环境标记（避免不必要的配置加载）
os.environ['DOCKER_ENV'] = 'true'

print("✅ 环境变量配置完成")
print("\n配置摘要:")
print(f"  Redis: {os.environ['REDIS_HOST']}:{os.environ['REDIS_PORT']}")
print(f"  MySQL: {os.environ['DB_HOST']}:{os.environ['DB_PORT']}")
print(f"  Neo4j: {os.environ['NEO4J_URI']}")
print(f"  Worker: {os.environ['KG_WORKER_NODE_NAME']}")
print(f"  进程数: {os.environ['KG_WORKERS_PER_PROVIDER']} 进程/Provider")

## 4. 验证网络连通性（可选）

运行此单元格验证能否连接到 Redis、MySQL、Neo4j

In [ ]:
import socket

def test_connection(host, port, name):
    """测试网络连通性"""
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(5)
        result = sock.connect_ex((host, int(port)))
        sock.close()
        if result == 0:
            print(f"✅ {name} 连接成功: {host}:{port}")
            return True
        else:
            print(f"❌ {name} 连接失败: {host}:{port}")
            return False
    except Exception as e:
        print(f"❌ {name} 连接错误: {e}")
        return False

print("测试网络连通性...\n")

redis_ok = test_connection(os.environ['REDIS_HOST'], os.environ['REDIS_PORT'], 'Redis')
mysql_ok = test_connection(os.environ['DB_HOST'], os.environ['DB_PORT'], 'MySQL')

# Neo4j URI 解析
neo4j_uri = os.environ['NEO4J_URI']
neo4j_host = neo4j_uri.split('://')[1].split(':')[0]
neo4j_port = neo4j_uri.split(':')[-1].split('/')[0]
neo4j_ok = test_connection(neo4j_host, neo4j_port, 'Neo4j')

print("\n网络连通性测试完成")
if redis_ok and mysql_ok and neo4j_ok:
    print("✅ 所有服务连接正常，可以启动 Worker")
else:
    print("⚠️ 部分服务连接失败，请检查配置")

## 5. 创建必要的目录

In [ ]:
import os

# 创建日志目录
!mkdir -p logs

print("✅ 目录创建完成")
print(f"日志目录: {os.path.abspath('logs')}")

## 6. 查看系统资源

查看 Kaggle 环境提供的资源

In [ ]:
import psutil

# CPU 信息
cpu_count = psutil.cpu_count()
cpu_percent = psutil.cpu_percent(interval=1)

# 内存信息
mem = psutil.virtual_memory()
mem_total = mem.total / (1024**3)  # GB
mem_available = mem.available / (1024**3)
mem_percent = mem.percent

# 磁盘信息
disk = psutil.disk_usage('/')
disk_total = disk.total / (1024**3)
disk_free = disk.free / (1024**3)
disk_percent = disk.percent

print("📊 Kaggle 环境资源信息")
print("=" * 50)
print(f"CPU: {cpu_count} 核")
print(f"CPU 使用率: {cpu_percent}%")
print(f"\n内存: {mem_total:.1f} GB")
print(f"可用内存: {mem_available:.1f} GB")
print(f"内存使用率: {mem_percent}%")
print(f"\n磁盘: {disk_total:.1f} GB")
print(f"可用磁盘: {disk_free:.1f} GB")
print(f"磁盘使用率: {disk_percent}%")
print("=" * 50)

# 估算可运行的 Worker 进程数
max_workers = int(mem_available * 1024 / 100)  # 每进程 100MB
print(f"\n💡 预计可运行 Worker 进程数: ~{max_workers} 个")

## 7. 启动 Worker 节点

**⚠️ 注意**: 
- 此单元格会持续运行直到手动停止或达到 9 小时限制
- 可以通过点击单元格左侧的停止按钮来停止 Worker
- 建议在新窗口打开 Kaggle 页面，避免意外关闭

In [ ]:
# 启动 Worker
!python worker.py

## 8. 监控 Worker 状态（可选）

如果需要在后台运行 Worker 并监控状态，可以使用此单元格

In [ ]:
import subprocess
import time
import psutil

# 后台启动 Worker
print("在后台启动 Worker...")
worker_process = subprocess.Popen(
    ['python', 'worker.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

print(f"✅ Worker 进程已启动 (PID: {worker_process.pid})")
print("\n开始监控资源使用...")
print("按 Interrupt 键停止监控\n")

try:
    while True:
        # CPU 使用率
        cpu = psutil.cpu_percent(interval=1)
        
        # 内存使用
        mem = psutil.virtual_memory()
        mem_used = mem.used / (1024**3)
        mem_total = mem.total / (1024**3)
        mem_percent = mem.percent
        
        # 进程数
        worker_processes = [p for p in psutil.process_iter(['name']) if 'python' in p.info['name'].lower()]
        
        # 输出状态
        print(f"\r[{time.strftime('%H:%M:%S')}] "
              f"CPU: {cpu:5.1f}% | "
              f"内存: {mem_used:5.1f}/{mem_total:.1f}GB ({mem_percent:5.1f}%) | "
              f"进程数: {len(worker_processes)}",
              end='', flush=True)
        
        time.sleep(5)
        
except KeyboardInterrupt:
    print("\n\n停止监控")
    worker_process.terminate()
    print("Worker 进程已停止")

## 9. 查看 Worker 日志（故障排查）

In [ ]:
# 查看最新的 50 行日志
!tail -n 50 logs/worker.log

## 10. 停止 Worker（清理）

In [ ]:
import signal
import os

# 查找并停止所有 worker.py 进程
!pkill -f "python worker.py"

print("✅ Worker 进程已停止")

# 验证
import time
time.sleep(2)
result = !ps aux | grep "worker.py" | grep -v grep | wc -l
if int(result[0]) == 0:
    print("✅ 所有 Worker 进程已清理")
else:
    print(f"⚠️ 还有 {result[0]} 个进程运行中")

---

## 📝 使用说明

### 准备工作

1. **打包项目代码**（本地执行）：
   ```bash
   cd chaishu-vue3
   zip -r chaishu-vue3.zip . -x "*.git*" "*node_modules*" "*frontend/dist*" "*__pycache__*" "*.pyc" "*venv*"
   ```

2. **上传到 Kaggle**（选择其一）：
   - 方式一: 在 Notebook 右侧 **+ Add Data** → **Upload** → 上传 ZIP
   - 方式二: 创建私有 Dataset，然后在 Notebook 中添加

### 首次使用

1. **上传代码**: 按上述方式上传 `chaishu-vue3.zip`
2. **修改配置**: 在 **Cell 3** 中填写 Redis/MySQL/Neo4j 连接信息
3. **依次运行**: Cell 1 → Cell 2 → ... → Cell 7
4. **查看日志**: Cell 7 会显示 Worker 运行日志

### 监控和管理

- **Cell 8**: 后台运行 Worker 并实时监控资源
- **Cell 9**: 查看日志排查问题
- **Cell 10**: 停止 Worker

### 注意事项

- ⚠️ **私有仓库**: 代码无法直接 git clone，必须手动上传
- ⚠️ **运行时长**: Kaggle 免费版最长运行 9 小时
- ⚠️ **保持在线**: 需要保持浏览器窗口打开
- ⚠️ **防止休眠**: 建议使用浏览器插件防止页面休眠
- ✅ **多账号轮换**: 可以使用多个 Kaggle 账号轮流运行

### 性能优化

- Kaggle CPU 环境: 4核 16GB 内存
- 推荐配置: 8-10 进程/Provider
- 总进程数建议: 40-80 个（取决于 Provider 数量）

### 故障排查

1. **代码上传问题**: 
   - 确认文件名为 `chaishu-vue3.zip`
   - 检查 Cell 2 是否成功解压
   - 查看 `worker.py` 是否存在

2. **网络连接问题**: 
   - 运行 Cell 4 检查连通性
   - 确认 Internet 设置为 ON

3. **配置错误**: 
   - 检查 Cell 3 的环境变量
   - 验证 Redis/MySQL/Neo4j 地址和密码

4. **进程崩溃**: 
   - 查看 Cell 9 的日志
   - 检查内存使用情况

---

## 📚 相关文档

- [Worker节点配置要求与部署指南](https://github.com/ronghuaxueleng/chaishu-vue3/blob/main/docs/Worker节点配置要求与部署指南.md)
- [Worker节点任务分配与故障恢复机制](https://github.com/ronghuaxueleng/chaishu-vue3/blob/main/docs/Worker节点任务分配与故障恢复机制.md)
- [Worker分布式部署实施总结](https://github.com/ronghuaxueleng/chaishu-vue3/blob/main/docs/Worker分布式部署实施总结.md)
- [Kaggle部署Worker节点指南](https://github.com/ronghuaxueleng/chaishu-vue3/blob/main/docs/Kaggle部署Worker节点指南.md)

---

**版本**: v1.1 (支持私有仓库)  
**更新日期**: 2025-10-17  
**作者**: Claude Code  
**项目**: [chaishu-vue3](https://github.com/ronghuaxueleng/chaishu-vue3)